In [1]:
#importing necessary library
from nsepy import get_history
import datetime as dt
from datetime import datetime,date
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from nsepy.derivatives import get_expiry_date

# **BUTTERFLY STRATEGY**

The Long Butterfly strategy is a directionless strategy that seeks profit from the time decay or decline in implied volatility(IV).

Long Butterfly is constructed using 3 strikes of fix interval, buying the lower strike option,selling two middle strike option, and buying the higher strike option.

Max profit = Strike difference - premium paid

Max Loss = Total premium paid.

Example of long Butterfly:

    Buy one lot of Nifty 16800 call at 190,

    Sell two lot of Nifty 17000 call at 130, # premium will be received

    Buy one lot of Nifty 17200 call at 90
    
Total premuim paid = 190 + 90 - (2*130) = 20

Total loss would be 20 only.

Now here trader is expecting that Nifty will close near 17000 so he sold 2 lots of Nifty call strike 17000 bought one lot Nifty 16800 call and 17200 call.

If Nifty remains in a range of 16800 - 17200 at expiry then trader will get maximum profit at 17000 that is 180(strike price difference - premium paid=200-20) and maximum loss will be 20(premium paid). 

**Long Butterfly strategy backtesting and Criteria**

Every first business day of the month trader is selling 200 points Nifty Butterfly of calls and holding his position till expiry.

In [2]:
# reading the csv file in which option prices are saved.
Table = pd.read_csv("Optionpivot.csv")
Table

,Date,Type,Expiry,9800.0,9900.0,10000.0,10100.0,10200.0,10300.0,10400.0,...,10700.0,10800.0,10900.0,11000.0,11100.0,11200.0,11300.0,11400.0,11500.0,Nifty
0,2018-01-01,CE,2018-01-25,NaN,NaN,497.00,407.45,322.80,248.00,178.00,...,41.30,21.60,10.20,5.60,2.90,2.30,1.30,1.10,0.70,10498.70
1,2018-01-01,PE,2018-01-25,NaN,NaN,20.00,29.00,44.30,65.25,95.00,...,253.00,332.95,413.90,510.05,586.85,690.45,802.85,847.00,1002.00,10498.70
2,2018-01-02,CE,2018-01-25,NaN,NaN,476.00,387.20,305.70,231.00,163.05,...,34.10,16.60,8.10,4.80,3.00,2.10,1.30,0.90,0.70,10472.20
3,2018-01-02,PE,2018-01-25,NaN,NaN,22.20,33.00,49.00,72.50,104.85,...,269.30,352.85,433.25,533.80,623.00,739.00,849.50,930.85,1024.10,10472.20
4,2018-01-03,CE,2018-01-25,NaN,NaN,478.00,390.00,309.35,228.95,163.00,...,32.75,16.00,7.35,4.50,2.60,2.00,1.35,0.95,0.80,10469.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2018-02-20,PE,2018-02-22,1.75,2.65,4.40,7.90,16.45,37.80,84.65,...,356.00,451.95,558.75,659.00,753.90,850.00,953.00,1030.00,1147.95,10343.75
70,2018-02-21,CE,2018-02-22,585.00,486.25,383.55,281.95,188.50,93.90,29.70,...,0.35,0.40,0.40,0.30,0.25,0.30,0.35,0.10,0.15,10392.90
71,2018-02-21,PE,2018-02-22,0.60,0.40,0.50,0.65,1.50,9.00,37.60,...,302.65,402.05,509.95,606.75,700.45,805.00,905.40,996.00,1099.05,10392.90
72,2018-02-22,CE,2018-02-22,577.50,469.75,374.00,280.55,180.40,75.90,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,10381.80


In [3]:
# abstracting month from date column

Table["month"] = pd.to_datetime(Table["Date"]).dt.strftime("%m")

In [4]:
Table.head()

,Date,Type,Expiry,9800.0,9900.0,10000.0,10100.0,10200.0,10300.0,10400.0,...,10800.0,10900.0,11000.0,11100.0,11200.0,11300.0,11400.0,11500.0,Nifty,month
0,2018-01-01,CE,2018-01-25,NaN,NaN,497.0,407.45,322.80,248.00,178.00,...,21.60,10.20,5.60,2.90,2.30,1.30,1.10,0.7,10498.7,01
1,2018-01-01,PE,2018-01-25,NaN,NaN,20.0,29.00,44.30,65.25,95.00,...,332.95,413.90,510.05,586.85,690.45,802.85,847.00,1002.0,10498.7,01
2,2018-01-02,CE,2018-01-25,NaN,NaN,476.0,387.20,305.70,231.00,163.05,...,16.60,8.10,4.80,3.00,2.10,1.30,0.90,0.7,10472.2,01
3,2018-01-02,PE,2018-01-25,NaN,NaN,22.2,33.00,49.00,72.50,104.85,...,352.85,433.25,533.80,623.00,739.00,849.50,930.85,1024.1,10472.2,01
4,2018-01-03,CE,2018-01-25,NaN,NaN,478.0,390.00,309.35,228.95,163.00,...,16.00,7.35,4.50,2.60,2.00,1.35,0.95,0.8,10469.4,01


In [6]:
# grouping dataframe by month
Table1 = Table.groupby("month").first()
Table1.index

Index(['01', '02'], dtype='object', name='month')

In [7]:
# abstracting month from date column by first converting into datetime object then abstracting it.
Table1["M"] = pd.to_datetime(Table1["Date"]).dt.strftime("%m")
Table1.M[1]

'02'

In [8]:
# calculating the ATM strike price on the first working day of the month on the date strategy will be executed.
Table1["ATM_Strike"] = round(Table1["Nifty"]/100)*100
Table1

,Date,Type,Expiry,9800.0,9900.0,10000.0,10100.0,10200.0,10300.0,10400.0,...,10900.0,11000.0,11100.0,11200.0,11300.0,11400.0,11500.0,Nifty,M,ATM_Strike
month,,,,,,,,,,,,,,,,,,,,,
01,2018-01-01,CE,2018-01-25,1333.95,1234.00,497.00,407.45,322.8,248.0,178.0,...,10.20,5.6,2.90,2.3,1.30,1.10,0.7,10498.7,01,10500.0
02,2018-02-01,CE,2018-02-22,1260.00,1080.45,1026.15,934.80,841.8,735.0,645.0,...,212.85,150.9,100.75,62.0,36.25,19.55,12.0,11031.3,02,11000.0


In [9]:
#creating the dataframe with the name table3
Table3 = pd.DataFrame({"Date":["0"],"Close":["0"],"Expiry":["0"],"Type":["0"],"month":["0"],"ATM":0,"OTM":0,"ITM":0,"Butterfly":0,"P&L":0
                   })
Table3

,Date,Close,Expiry,Type,month,ATM,OTM,ITM,Butterfly,P&L
0,0,0,0,0,0,0,0,0,0,0


In [10]:
#creating the dataframe with the name table4

Table4 = pd.DataFrame({"Date":["0"],"Close":["0"],"Expiry":["0"],"Type":["0"],"month":["0"],"ATM":0,"OTM":0,"ITM":0,"Butterfly":0,"P&L":0
                   })
Table4

,Date,Close,Expiry,Type,month,ATM,OTM,ITM,Butterfly,P&L
0,0,0,0,0,0,0,0,0,0,0


In [11]:
# Monthly date wise prices of option strikes are fetched in table2 through for loop and appended into table 3 and table4 is appended with table2.
for x in range(0,2):
    e = Table1.M[x] # fetching month
    d = Table1.Expiry[x] # fetching expiry date
    a = float((Table1.ATM_Strike[x])) # coverting and fetching ATM strike
    b = a + 200  # otm strike
    c = a - 200  #itm strike
    f1 = str(a) 
    f2 = str(b) 
    f3 = str(c) 
    Table2 = pd.DataFrame({"Date":Table["Date"],"Close":Table["Nifty"],"Expiry":Table["Expiry"],"Type":Table["Type"],"month":Table["month"],
                        "ATM":Table[f1],"OTM":Table[f2],"ITM":Table[f3]})
    Table2 = Table2.where((Table2.month == e) & (Table2.Expiry == d) & (Table2.Type =="CE"))
    Table2 = Table2.dropna()
    Table2["Butterfly"] = (Table2["ATM"]*2)-Table2["ITM"] - Table2["OTM"]
    Table2["P&L"] = Table2["Butterfly"].diff().cumsum()
    Table3 = Table3.append(Table2)
    Table4 = Table4.append(Table2.where((Table2.Date == d)))
    Table4 = Table4.dropna()
Table4

,Date,Close,Expiry,Type,month,ATM,OTM,ITM,Butterfly,P&L
0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00
36,2018-01-25,11065.1,2018-01-25,CE,01,560.50,367.55,767.00,-13.55,34.15
72,2018-02-22,10381.8,2018-02-22,CE,02,0.05,0.05,0.05,0.00,50.20


In [12]:
Table4.columns

Index(['Date', 'Close', 'Expiry', 'Type', 'month', 'ATM', 'OTM', 'ITM',
       'Butterfly', 'P&L'],
      dtype='object')

In [13]:
#dropping of the respective columns from the datafram table4 only Month and P&L will be there.
Table4.drop(["Date","Close","Expiry","Type","ATM","OTM","ITM","Butterfly"],axis=1)
Table4["P&L"].cumsum()
Table4["TotalP&L"] = Table4["P&L"].cumsum()
    
    

In [16]:
Table4

,Date,Close,Expiry,Type,month,ATM,OTM,ITM,Butterfly,P&L,TotalP&L
0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
36,2018-01-25,11065.1,2018-01-25,CE,01,560.50,367.55,767.00,-13.55,34.15,34.15
72,2018-02-22,10381.8,2018-02-22,CE,02,0.05,0.05,0.05,0.00,50.20,84.35


**If we see output of Table4, we can see that in first month of the strategy we earned total profit of 2550(34\*75) and in second month we earned 3750(50*75).**

In [14]:
# Converting Table3 dataframe into csv file.
Table3.to_csv("Daywise_ButterflyStrategy_computation.csv",index=True,encoding="utf8")

In [15]:
pd.read_csv("Daywise_ButterflyStrategy_computation.csv")

,Unnamed: 0,Date,Close,Expiry,Type,month,ATM,OTM,ITM,Butterfly,P&L
0,0,0,0.00,0,0,0,0.00,0.00,0.00,0.000000e+00,0.00
1,0,2018-01-01,10498.70,2018-01-25,CE,1,120.80,41.30,248.00,-4.770000e+01,NaN
2,2,2018-01-02,10472.20,2018-01-25,CE,1,106.80,34.10,231.00,-5.150000e+01,-3.80
3,4,2018-01-03,10469.40,2018-01-25,CE,1,105.00,32.75,228.95,-5.170000e+01,-4.00
4,6,2018-01-04,10526.20,2018-01-25,CE,1,132.35,41.95,273.25,-5.050000e+01,-2.80
5,8,2018-01-05,10573.20,2018-01-25,CE,1,151.10,48.00,307.00,-5.280000e+01,-5.10
6,10,2018-01-08,10631.40,2018-01-25,CE,1,176.90,59.00,345.00,-5.020000e+01,-2.50
7,12,2018-01-09,10646.90,2018-01-25,CE,1,188.60,61.55,357.55,-4.190000e+01,5.80
8,14,2018-01-10,10637.05,2018-01-25,CE,1,183.20,57.20,355.00,-4.580000e+01,1.90
9,16,2018-01-11,10654.05,2018-01-25,CE,1,197.00,64.00,369.30,-3.930000e+01,8.40
